In [ ]:
from models.ABrain import Airfoil_Brain
import numpy as np
import matplotlib.pyplot as plt
import torch

# Initialize
current_ada_iter=65
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
airfoil_brain=Airfoil_Brain(model_path_header='./model_save/pp_model_save/ada{:d}'.format(current_ada_iter), device=device) # load prior model
airfoil_brain.std_var='uniform'

In [ ]:
# Define the grids
Ma_grid=np.array([0.4, 0.7])
AoA_grid=np.linspace(-20,25,101)
Re_base_grid = np.array([5e6, 5e6])

# Create meshgrid
Ma, AoA, Re_base = np.meshgrid(Ma_grid, AoA_grid, Re_base_grid, indexing='ij')

# Flatten the grids
Ma_flat = Ma.flatten()
AoA_flat = AoA.flatten()
Re_base_flat = Re_base.flatten()

# Combine into a single array with the shape (num, 3)
combined_grid = np.stack((Ma_flat, AoA_flat, Re_base_flat), axis=-1)

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

import ipywidgets as widgets
from IPython.display import display


# Create a subplot figure
fig = make_subplots(rows=2, cols=3, subplot_titles=('Generated Airfoil',' ', ' ', 'Lift Coefficient', 'Drag Coefficient', 'Momment Coefficient'), vertical_spacing=0.15)

# Airfoil Plot
def func_contour_plotly(x, y):
    trace = go.Scatter(
        x=x, 
        y=y, 
        mode='lines+markers', 
        name='Generated airfoil',
        line=dict(
            color='black',  # Set line color
            dash='solid'  # Set line style, options include: 'dash', 'dot', 'dashdot', 'solid', etc.
        ),
        marker=dict(
            color='black'  # Set marker color
        )
        )
    return trace

# Camber and Thickness Plot
def func_contour2_plotly(AoA, c81, upper, lower, color='black', label=None, show_legend=False):
    trace = go.Scatter(
        x=AoA,
        y=c81,
        mode='lines+markers',
        name=label,
        line=dict(
            color=color,  # Set line color for Camber
            dash='solid'  # Set line style
        ),
        marker=dict(
            symbol='circle',
            color='rgba(0,0,0,0)',          # transparent face color
            line=dict(color=color, width=1),  # black edge color
            size=7
        ),
        showlegend=show_legend
        )
    
    # Upper boundary line (transparent)
    fill_trace0 = go.Scatter(
        x=AoA,  
        y=upper,
        name=None,
        fill=None,  # No fill
        mode='lines',
        # line=dict(color='rgba(0,0,0,0)'),  # Transparent line color
        line=dict(color=color, width=0.5),  # Transparent line color
        showlegend=show_legend
    )

    # Lower boundary line with fill to the upper boundary
    fill_trace1 = go.Scatter(
        x=AoA,  
        y=lower, 
        name='CI' ,
        fill='tonexty',  # Fill to the next trace in y direction
        mode='lines',
        # line=dict(color='rgba(0,0,0,0)'),  # Transparent line color
        line=dict(color=color, width=0.5),  # Transparent line color
    
        fillcolor='rgba' + str(tuple(int(color.strip('#')[i:i+2], 16) for i in (0, 2, 4)) + (0.2,)),  # Correctly set fill color with alpha
        showlegend=show_legend
    )
    
    

    return trace, fill_trace0, fill_trace1
    stamp0=time.time()

input_dims=10
ds_size=5
z_lb=0
z_ub=1

latent_geom=np.ones((10))*0.5
latent_geom_expand=[]
for i in range(combined_grid.shape[0]):
    latent_geom_expand.append(np.concatenate([latent_geom, combined_grid[i]]))
latent_geom_expand=np.array(latent_geom_expand)

airfoils, c81_mean, c81_std, c81_std2 = airfoil_brain.evaluate(latent_geom_expand)
c81_mean=c81_mean.reshape(2,AoA_grid.shape[0],2,3)
c81_std=c81_std.reshape(2,AoA_grid.shape[0],2,3)
c81_std2[0]=c81_std2[0].reshape(2,AoA_grid.shape[0],2,3)
c81_std2[1]=c81_std2[1].reshape(2,AoA_grid.shape[0],2,3)
# Initial data plots

fig.add_trace(func_contour_plotly(airfoils[0,:,0], airfoils[0,:,1]), row=1, col=[1,2,3])
fig.update_layout(
    font=dict(
        family="Times New Roman",  # Font family
        size=15,                   # Font size
        color="black"              # Font color
    ),
    
    title_font=dict(
        family="Times New Roman",
        size=18,
        color="#000000"
    ),
    
    plot_bgcolor='white',  # Background color of the plotting area
    
    xaxis=dict(
        title="X",
        range=[-0.1, 1.1],
        showgrid=True,
        ticks="inside",
        tickvals=np.arange(-0.1,1.1,0.1).tolist(),
        domain=[0,1]
    ),
    
    yaxis=dict(
        title="Y",
        range=[-0.35, 0.35],
        showgrid=True,
        ticks="inside",
        tickvals=np.arange(-0.35,0.35,0.1).tolist()  
    ),
)

# Performance plots (subplot axes)
axis_settings = {
    'xaxis4': dict(range=[-20, 25], tickvals=np.arange(-20, 26, 5).tolist()),
    'yaxis4': dict(range=[-1.5, 2.0], tickvals=np.arange(-1.5, 2.1, 0.5).tolist()),
    'xaxis5': dict(range=[-20, 25], tickvals=np.arange(-20, 26, 5).tolist()),
    'yaxis5': dict(range=[0.00, 0.1], tickvals=np.linspace(0.00, 0.1, 5).tolist()),
    'xaxis6': dict(range=[-20, 25], tickvals=np.arange(-20, 26, 5).tolist()),
    'yaxis6': dict(range=[-0.25, 0.15], tickvals=np.arange(-0.25, 0.16, 0.1).tolist())
}

for axis_name, settings in axis_settings.items():
    fig['layout'][axis_name].update(
        showgrid=True,
        ticks="inside",
        **settings
    )

# Global axis style
fig.update_xaxes(mirror=True, showline=True, linecolor='black', gridcolor='lightgrey')
fig.update_yaxes(mirror=True, showline=True, linecolor='black', gridcolor='lightgrey')

colors = ['#ff0000', '#00008B', '#006400']
ma_seq=[0,1]
ci_scale=2
for j in range(3):
    for i in range(2):
        if j==1:
            traces = func_contour2_plotly(AoA_grid, 10**(c81_mean[i,:,0,j]),10**(c81_mean[i,:,0,j]+ci_scale*c81_std[i,:,0,j]),\
                10**(c81_mean[i,:,0,j]-ci_scale*c81_std[i,:,0,j]), color=colors[i], show_legend=True)
        else:
            traces = func_contour2_plotly(AoA_grid, c81_mean[i,:,0,j], c81_mean[i,:,0,j]+ci_scale*c81_std[i,:,0,j],\
                c81_mean[i,:,0,j]-ci_scale*c81_std[i,:,0,j], color=colors[i], label='Ma {:.1f}'.format(Ma_grid[ma_seq[i]]))
        fig.add_trace(traces[0], row=2, col=j+1)
        fig.add_trace(traces[1], row=2, col=j+1)
        fig.add_trace(traces[2], row=2, col=j+1)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig_widget = go.FigureWidget(fig)

def update_fig(**kwargs):
    with fig_widget.batch_update():

        input_dims=10
        point=np.zeros((1,input_dims))
        for i in range(input_dims):
            name = f'Z{i+1}'
            point[0,i]=kwargs[name]

   
        latent_geom=point[0].copy()
        latent_geom_expand=[]
        for i in range(combined_grid.shape[0]):
            latent_geom_expand.append(np.concatenate([latent_geom, combined_grid[i]]))
        latent_geom_expand=np.array(latent_geom_expand)

        airfoils, c81_mean, c81_std, c81_std2 = airfoil_brain.evaluate(latent_geom_expand)
        c81_mean=c81_mean.reshape(2,AoA_grid.shape[0],2,3)
        c81_std=c81_std.reshape(2,AoA_grid.shape[0],2,3)
        
        c81_std2_ep=c81_std2[0].reshape(2,AoA_grid.shape[0],2,3)
        c81_std2_al=c81_std2[1].reshape(2,AoA_grid.shape[0],2,3)

        fig_widget.data[0].x = airfoils[0,:,0]
        fig_widget.data[0].y = airfoils[0,:,1]
        
        p=0
        
        for j in range(3):
            for i in range(2):
                mean_temp=c81_mean[i,:,0,j]
                
                # std_temp=np.clip(c81_std2_ep[i,:,0,j]-c81_std2_al[i,:,0,j]*0.5,0,False)*1e10
                
                # std_temp=c81_std2[0][i,:,0,j]
                std_temp=c81_std[i,:,0,j]
                if j ==1:
                    
                
                    # Update the Camber trace
                    fig_widget.data[p+1].x = AoA_grid
                    fig_widget.data[p+1].y = 10**(mean_temp)
                    fig_widget.data[p+2].x = AoA_grid 
                    fig_widget.data[p+2].y = 10**(mean_temp+ci_scale*std_temp)
                    fig_widget.data[p+3].x = AoA_grid
                    fig_widget.data[p+3].y = 10**(mean_temp-ci_scale*std_temp)
                else:
                    fig_widget.data[p+1].x = AoA_grid
                    fig_widget.data[p+1].y = mean_temp
                    fig_widget.data[p+2].x = AoA_grid 
                    fig_widget.data[p+2].y = mean_temp+ci_scale*std_temp 
                    fig_widget.data[p+3].x = AoA_grid
                    fig_widget.data[p+3].y = mean_temp-ci_scale*std_temp 
                p+=3
            


# Create initial figure widget
fig_widget = go.FigureWidget(fig)

# Create sliders
sliders = []
for i in range(input_dims):
    
    name = f'Z{i+1}'

    slider = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        description=name,
        continuous_update=True,  # This ensures the update function is called only when you release the slider
        step=0.001
    )
    slider.layout.height='50px'
    
    sliders.append(slider)

# Link sliders to the update function
for slider in sliders:
    slider.observe(lambda change: update_fig(**{s.description: s.value for s in sliders}), 'value')

# Create a vertical box for sliders
vbox_sliders = widgets.VBox(sliders)

# Create a horizontal box to position the sliders on the left and the figure on the right
fig_widget.update_layout(
    height=700,
    margin=dict(l=20, r=50, b=50, t=50)
)
# layout = widgets.HBox([vbox_sliders, fig_widget])
layout = widgets.GridBox(children=[vbox_sliders, fig_widget], 
                         layout=widgets.Layout(grid_template_columns="30% 70%"))
# Display the final layout



In [ ]:
display(layout)